# Demo for Classification Model Task

The current notebook is going to be a demo of the classification model capabilities, while also being used as the initial steps for the creation of the pipeline of the deployed version.

In [18]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import pickle, joblib
sys.path.insert(0, '..')

import support_functions

## Simple Mock Gait Cycle data from Gyroscope

In [19]:

def simulate_custom_gait_signal(duration_sec: int=60 , fs: int=100, freq: int=1.2, stroke: bool=False, noise_std: float=0.05):
    '''
    Simulate custom gait signals for stroke and healthy individuals.
    Args:
        duration_sec (int): Duration of the signal in seconds.
        fs (int): Sampling frequency in Hz.
        freq (float): Frequency of the sine wave.
        stroke (bool): If True, simulate stroke condition.
        noise_std (float): Standard deviation of the noise.
    '''
    t = np.linspace(0, duration_sec, int(duration_sec * fs))
    
    # Define target peak values
    if stroke:
        max_val   = 300
        amp_right = 200
    else:
        max_val   = 450
        amp_right = 420
    
    # Simulate left and right signals (right affected in stroke)
    left_z  = (max_val / 2) * np.sin(2 * np.pi * freq * t) + np.random.normal(0, noise_std * max_val, size=t.shape)
    right_z = (amp_right / 2) * np.sin(2 * np.pi * freq * t + (0.2 if stroke else 0)) + np.random.normal(0, noise_std * amp_right, size=t.shape)

    # Return DataFrame
    dataframe = pd.DataFrame({
        'timestamp (+0700)': pd.date_range('2024-01-01 00:00:00', periods=len(t), freq=f'{int(1000/fs)}ms'),
        'left-z-axis (deg/s)': left_z,
        'right-z-axis (deg/s)': right_z
    })
    return dataframe

## Load the Linear Regression Classification Model 

In [20]:
# Load the model and the scaler
model = joblib.load('/Users/stamatiosorphanos/Documents/GaitRehab/models/classification_logistic_regression.pkl')
scaler = joblib.load('/Users/stamatiosorphanos/Documents/GaitRehab/models/classification_scaler.pkl')

print('Input features:', model.n_features_in_)
print('Classes:', model.classes_)

Input features: 4
Classes: [0 1]


### Healthy Subject Prediction 

In [21]:
# Create the mock raw gyroscope data, get the features we use during the training and scale them for prediction
healthy_df = simulate_custom_gait_signal(stroke=False)
print('The mock raw Gyroscope data is:', healthy_df.head())

# Get the features used for training min, max for the z-axis of the gyroscope
raw_features = support_functions.time_domain_features(healthy_df)
print("\nRaw input features:", raw_features)


# Scale the features
raw_dict = {'right-z-axis-(deg/s)-min': [raw_features[0]], 'left-z-axis-(deg/s)-min' : [raw_features[1]], 
            'right-z-axis-(deg/s)-max': [raw_features[2]], 'left-z-axis-(deg/s)-max' : [raw_features[3]]}
raw_data = pd.DataFrame(raw_dict)
X_input = scaler.transform(raw_data)
print("\nScaled input features:", X_input)

# Predict the class and the probability of asymmetry
y_pred = model.predict(X_input)[0]
y_prob = model.predict_proba(X_input)[0][1] 

print(f'Predicted class: {y_pred} — Probability of asymmetry: {y_prob:.2f} - Probability of symmetry: {1-y_prob:.2f}')

The mock raw Gyroscope data is:         timestamp (+0700)  left-z-axis (deg/s)  right-z-axis (deg/s)
0 2024-01-01 00:00:00.000             7.463139            -22.224477
1 2024-01-01 00:00:00.010           -12.254010             12.756597
2 2024-01-01 00:00:00.020            19.431611             42.267329
3 2024-01-01 00:00:00.030            60.264337             -0.391846
4 2024-01-01 00:00:00.040            80.334248              6.479257

Raw input features: [-269.3534933420498, -288.18783926502584, 282.80648182675566, 290.1240077285966]

Scaled input features: [[-0.71240051 -0.93527306 -0.47841718 -0.7406694 ]]
Predicted class: 1 — Probability of asymmetry: 0.50 - Probability of symmetry: 0.50


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))
plt.plot(healthy_df["left-z-axis (deg/s)"], label="Left Z")
plt.plot(healthy_df["right-z-axis (deg/s)"], label="Right Z")
plt.title(f"Prediction: {'Asymmetric' if y_pred == 1 else 'Symmetric'} (prob={y_prob:.2f})")
plt.xlabel("Time (samples)")
plt.ylabel("Deg/s")
plt.legend()
plt.grid(True)
plt.show()

### Stroke Subject Prediction 

In [22]:
# Create the mock raw gyroscope data, get the features we use during the training and scale them for prediction
stroke_df = simulate_custom_gait_signal(stroke=True)
print('The mock raw Gyroscope data is:', stroke_df.head())

# Get the features used for training min, max for the z-axis of the gyroscope
raw_features = support_functions.time_domain_features(stroke_df)
print("\nRaw input features:", raw_features)


# Scale the features
raw_dict = {'right-z-axis-(deg/s)-min': [raw_features[0]], 'left-z-axis-(deg/s)-min' : [raw_features[1]], 
            'right-z-axis-(deg/s)-max': [raw_features[2]], 'left-z-axis-(deg/s)-max' : [raw_features[3]]}
raw_data = pd.DataFrame(raw_dict)
X_input = scaler.transform(raw_data)
print("\nScaled input features:", X_input)

# Predict the class and the probability of asymmetry
y_pred = model.predict(X_input)[0]
y_prob = model.predict_proba(X_input)[0][1] 

print(f'Predicted class: {y_pred} — Probability of asymmetry: {y_prob:.2f} - Probability of symmetry: {1-y_prob:.2f}')

The mock raw Gyroscope data is:         timestamp (+0700)  left-z-axis (deg/s)  right-z-axis (deg/s)
0 2024-01-01 00:00:00.000             9.377646             13.901287
1 2024-01-01 00:00:00.010            -3.657342             27.238482
2 2024-01-01 00:00:00.020            13.430461             36.780463
3 2024-01-01 00:00:00.030            31.500940             21.393730
4 2024-01-01 00:00:00.040            13.536458             62.375495

Raw input features: [-123.81779878179297, -196.17729043786903, 130.15865550916763, 199.38953768711676]

Scaled input features: [[ 1.08657775  0.32807282 -1.91711927 -1.83344322]]
Predicted class: 1 — Probability of asymmetry: 0.99 - Probability of symmetry: 0.01
